In [220]:
%run ./utility.ipynb

In [221]:
df = pd.read_csv('./data/hmeq.csv')
df.shape

(5960, 13)

# Index
1. [Drop empty rows](#Drop-empty-rows)
2. [Get dummies](#Get-dummies)
3. [Drop correlated](#Drop-correlated)
4. [Train test split](#Train-test-split)
5. [Imputing](#Imputing)
6. [Unskewing](#Unskewing)
7. [Scale data](#scale-data)
8. [Balance data](#balance-data)
9. [Saving cleaned data](#Saving-cleaned-datasets)

# Drop empty rows

The data analysis showed that rows 3 and 1405 where completely empty

In [222]:
no_empty_rows_df = df.copy()
print(no_empty_rows_df.iloc[[3, 1405]])
no_empty_rows_df.drop([3, 1405], axis=0, inplace=True)
no_empty_rows_df.iloc[[3, 1405]]

      BAD   LOAN  MORTDUE  VALUE REASON  JOB  YOJ  DEROG  DELINQ  CLAGE  NINQ  \
3       1   1500      NaN    NaN    NaN  NaN  NaN    NaN     NaN    NaN   NaN   
1405    0  10800      NaN    NaN    NaN  NaN  NaN    NaN     NaN    NaN   NaN   

      CLNO  DEBTINC  
3      NaN      NaN  
1405   NaN      NaN  


,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN
1407,1,10800,52600.0,66700.0,DebtCon,ProfExe,2.0,0.0,0.0,110.566667,5.0,26.0,NaN


# Get dummies

In the data analysis we saw that JOB and REASON where categorical features so I'll first make dummy variables for these. I'll also add a new category "missing" to be used as an imputation for the null values so we don't lose the information of them being null.

In [223]:
dummy_var_df = no_empty_rows_df.copy()

print(f"# missing values JOB: {dummy_var_df['JOB'].isnull().sum()}")
dummy_var_df['JOB'] = dummy_var_df['JOB'].fillna("missing")
print(f"# missing values JOB: {dummy_var_df['JOB'].isnull().sum()}")

print(f"# missing values REASON: {dummy_var_df['REASON'].isnull().sum()}")
dummy_var_df['REASON'] = dummy_var_df['REASON'].fillna("missing")
print(f"# missing values REASON: {dummy_var_df['REASON'].isnull().sum()}")

job_dummies = pd.get_dummies(dummy_var_df['JOB'], prefix="JOB")
reason_dummies = pd.get_dummies(dummy_var_df['REASON'], prefix="REASON")

dummy_var_df = pd.concat([dummy_var_df, job_dummies, reason_dummies], axis=1)
dummy_var_df.drop(["JOB", "REASON"], axis=1, inplace=True)
dummy_var_df.head()

# missing values JOB: 277
# missing values JOB: 0
# missing values REASON: 250
# missing values REASON: 0


,BAD,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,...,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self,JOB_missing,REASON_DebtCon,REASON_HomeImp,REASON_missing
0,1,1100,25860.0,39025.0,10.5,0.0,0.0,94.366667,1.0,9.0,...,False,False,True,False,False,False,False,False,True,False
1,1,1300,70053.0,68400.0,7.0,0.0,2.0,121.833333,0.0,14.0,...,False,False,True,False,False,False,False,False,True,False
2,1,1500,13500.0,16700.0,4.0,0.0,0.0,149.466667,1.0,10.0,...,False,False,True,False,False,False,False,False,True,False
4,0,1700,97800.0,112000.0,3.0,0.0,0.0,93.333333,0.0,14.0,...,False,True,False,False,False,False,False,False,True,False
5,1,1700,30548.0,40320.0,9.0,0.0,0.0,101.466002,1.0,8.0,...,False,False,True,False,False,False,False,False,True,False


# Drop correlated

MORTDUE and VALUE had very high correlation (88%), meaning that they produce a lot of redundant information so I'll drop MORTDUE

In [224]:
drop_mortdue_df = dummy_var_df.copy()

drop_mortdue_df.drop("MORTDUE", axis=1, inplace=True)
drop_mortdue_df.columns

Index(['BAD', 'LOAN', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ',
       'CLNO', 'DEBTINC', 'JOB_Mgr', 'JOB_Office', 'JOB_Other', 'JOB_ProfExe',
       'JOB_Sales', 'JOB_Self', 'JOB_missing', 'REASON_DebtCon',
       'REASON_HomeImp', 'REASON_missing'],
      dtype='object')

# Train test split

Before we can balance, impute, scale and unskew our data we need to first split it in a train and test set to prevent information leakage

In [225]:
X = drop_mortdue_df.drop("BAD", axis=1)
y = drop_mortdue_df["BAD"]

print(f"Total num of instances: {drop_mortdue_df.shape[0]}, total num of features: {drop_mortdue_df.shape[1]}")
print(f"num of ind feature instances: {X.shape[0]}, num of ind feature features: {X.shape[1]}")
print(f"num of dep feature instances: {y.shape[0]}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

print(f"num of training instances: {X_train.shape[0]}, num of training features: {X.shape[1]}")
print(f"num of test instances: {X_test.shape[0]}, num of test features: {X.shape[1]}")
print(f"num of train target instances: {y_train.shape[0]}")
print(f"num of test target: {y_test.shape[0]}")


Total num of instances: 5958, total num of features: 20
num of ind feature instances: 5958, num of ind feature features: 19
num of dep feature instances: 5958
num of training instances: 4766, num of training features: 19
num of test instances: 1192, num of test features: 19
num of train target instances: 4766
num of test target: 1192


# Imputing

In [226]:
imputed_X_train = X_train.copy()
imputed_X_test = X_test.copy()

In [227]:
#set credt lines that are younger then 12months as recent
train_indices = imputed_X_train[(imputed_X_train["NINQ"].isnull()) & -(imputed_X_train["CLNO"].isnull()) & (imputed_X_train["CLAGE"] < 12)][["CLAGE", "NINQ", "CLNO"]].index
imputed_X_train.loc[train_indices, "NINQ"] = imputed_X_train.loc[train_indices, "CLNO"]

test_indices = imputed_X_test[(imputed_X_test["NINQ"].isnull()) & -(imputed_X_test["CLNO"].isnull()) & (imputed_X_test["CLAGE"] < 12)][["CLAGE", "NINQ", "CLNO"]].index
imputed_X_test.loc[test_indices, "NINQ"] = imputed_X_test.loc[test_indices, "CLNO"]

print(imputed_X_test.loc[test_indices, ["CLAGE", "NINQ", "CLNO"]])
imputed_X_train.loc[train_indices, ["CLAGE", "NINQ", "CLNO"]]

         CLAGE  NINQ  CLNO
4845  5.243341  41.0  41.0
1091  0.507115   8.0   8.0
1572  4.412770   8.0   8.0


,CLAGE,NINQ,CLNO
1360,11.963733,7.0,7.0
691,9.100000,8.0,8.0
1122,3.044384,8.0,8.0
1083,2.820786,7.0,7.0
4758,10.133333,41.0,41.0
1347,9.534143,7.0,7.0


In [228]:
def impute(series):
    imp = IterativeImputer(max_iter=10, random_state=42)
    return imp.fit_transform(series.values.reshape(-1, 1))

In [229]:
columns = ["YOJ", "VALUE", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"]

for col in columns:
    imputed_X_train[col] = impute(imputed_X_train[col])
    imputed_X_test[col] = impute(imputed_X_test[col])


imputed_X_train.isnull().sum()

LOAN              0
VALUE             0
YOJ               0
DEROG             0
DELINQ            0
CLAGE             0
NINQ              0
CLNO              0
DEBTINC           0
JOB_Mgr           0
JOB_Office        0
JOB_Other         0
JOB_ProfExe       0
JOB_Sales         0
JOB_Self          0
JOB_missing       0
REASON_DebtCon    0
REASON_HomeImp    0
REASON_missing    0
dtype: int64

In [230]:
imputed_X_test.isnull().sum()

LOAN              0
VALUE             0
YOJ               0
DEROG             0
DELINQ            0
CLAGE             0
NINQ              0
CLNO              0
DEBTINC           0
JOB_Mgr           0
JOB_Office        0
JOB_Other         0
JOB_ProfExe       0
JOB_Sales         0
JOB_Self          0
JOB_missing       0
REASON_DebtCon    0
REASON_HomeImp    0
REASON_missing    0
dtype: int64

# Scale data

I'm going to be working with distance based algorithms thus all feature scales need to be the same

In [231]:
scaled_X_train = imputed_X_train.copy()
scaled_X_test = imputed_X_test.copy()

In [232]:
def scale_data(series):
    scaler = StandardScaler()
    return scaler.fit_transform(series.values.reshape(-1, 1))

In [233]:
na_cols = ['JOB_Mgr', 'JOB_Office', 'JOB_Other', 'JOB_ProfExe','JOB_Sales', 'JOB_Self', 'JOB_missing', 'REASON_DebtCon','REASON_HomeImp', 'REASON_missing']

for col in scaled_X_train.drop(na_cols, axis=1).columns:
    scaled_X_train[col] = scale_data(scaled_X_train[col])
    scaled_X_test[col] = scale_data(scaled_X_test[col])

# Saving cleaned datasets

In [239]:
scaled_X_train.to_csv('./data/cleaned_X_train.csv')
y_train.to_csv('./data/cleaned_y_train.csv')

scaled_X_test.to_csv('./data/cleaned_X_test.csv')
y_test.to_csv('./data/cleaned_y_test.csv')